In [1]:
from sympy import *
import math
import numpy as np

### 20.1

In [2]:
def PlaneBeamElementLength(x1,y1,x2,y2):
    return math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1))

In [3]:
def PlaneBeamElementStiffness(E,I,L):
    return (E*I)/(L**3)*np.array([[12, 6*L, -12, 6*L],
                                [6*L,4*L**2, -6*L, 2*L**2],
                                [-12, -6*L, 12, -6*L],
                                [6*L, 2*L**2,  -6*L,4*L**2]]) 

In [4]:
def PlaneBeamAssemble(K,k,i,j):
    # K - np.array
    # k - np.array
    K[2*i,2*i] = K[2*i,2*i] + k[0,0]
    K[2*i,2*i+1] = K[2*i,2*i+1] + k[0,1] 
    K[2*i,2*j] = K[2*i,2*j] + k[0,2] 
    K[2*i,2*j+1] = K[2*i,2*j+1] + k[0,3]
    K[2*i+1,2*i] = K[2*i+1,2*i] + k[1,0]
    K[2*i+1,2*i+1] = K[2*i+1,2*i+1] + k[1,1] 
    K[2*i+1,2*j] = K[2*i+1,2*j] + k[1,2] 
    K[2*i+1,2*j+1] = K[2*i+1,2*j+1] + k[1,3] 
    K[2*j,2*i] = K[2*j,2*i] + k[2,0] 
    K[2*j,2*i+1] = K[2*j,2*i+1] + k[2,1]
    K[2*j,2*j] = K[2*j,2*j] + k[2,2]
    K[2*j,2*j+1] = K[2*j,2*j+1] + k[2,3] 
    K[2*j+1,2*i] = K[2*j+1,2*i] + k[3,0] 
    K[2*j+1,2*i+1] = K[2*j+1,2*i+1] + k[3,1]
    K[2*j+1,2*j] = K[2*j+1,2*j] + k[3,2] 
    K[2*j+1,2*j+1] = K[2*j+1,2*j+1] + k[3,3]
        
    return K

In [5]:
def FEA_u(coord, elcon, bc_node, bc_val, global_force,  I, E):
    K=np.zeros(shape=(2*np.max(elcon)+2,2*np.max(elcon)+2))
    for el in elcon:
        L=PlaneBeamElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
        k=PlaneBeamElementStiffness(E, I, L)
        K=PlaneBeamAssemble(K,k,el[0],el[1])
    
    F = global_force
    
    bc=np.array([bc_node, 
                bc_val]).T
    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))

    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.lstsq(K[fdof,:][:,fdof],
                      (F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T, rcond=None)[0] 
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    
#     Q=np.dot(K,d).T-F 
    
    return d

In [6]:
I=8000
E=20e6

In [7]:
coord=np.array([0,0,
                400,0,
               1200,0]).reshape(3,2)

In [8]:
elcon=np.array([[0, 1],
                [1, 2]])

In [9]:
bc_node = [0,4]

In [10]:
bc_val=[ 0., 0.]

In [11]:
global_force=np.array([0,0,
                      -50000,0,
                       0, 0])

In [12]:
FEA_u(coord, elcon, bc_node, bc_val, global_force, I, E)

array([ 0.        , -0.02777778, -8.88888889, -0.01111111,  0.        ,
        0.02222222])

### 20.2

In [13]:
coord=np.array([0,0,
                400,0,
               1200,0]).reshape(3,2)

In [14]:
elcon=np.array([[0, 1],
                [1, 2]])

In [15]:
bc_node = [0, 1, 4,5]

In [16]:
bc_val=[0., 0., 0.,0]

In [17]:
global_force=np.array([0,0,
                       -50000,0,
                       0, 0])

In [18]:
FEA_u(coord, elcon, bc_node, bc_val, global_force, I, E)

array([ 0.        ,  0.        , -1.97530864, -0.0037037 ,  0.        ,
        0.        ])

### 20.4

In [19]:
coord=np.array([0,0,
                500,0]).reshape(2,2)

In [20]:
elcon=np.array([[0, 1]])

In [21]:
bc_node = [0, 1]

In [22]:
bc_val=[0., 0.]

In [23]:
global_force=np.array([0,0,
                       -50000,0])

In [24]:
FEA_u(coord, elcon, bc_node, bc_val, global_force, I, E)

array([  0.        ,   0.        , -13.02083333,  -0.0390625 ])

### 20.8

In [25]:
coord=np.array([0,0,
                500,0,
                1000,0]).reshape(3,2)

In [26]:
elcon=np.array([[0, 1],
                [1,2]])

In [27]:
bc_node = [0, 1, 4]

In [28]:
bc_val=[0., 0., 0.]

In [29]:
global_force=np.array([0,0,
                       -50000,0,
                      0,0])

In [30]:
FEA_u(coord, elcon, bc_node, bc_val, global_force, I, E)

array([ 0.00000000e+00,  0.00000000e+00, -2.84830729e+00, -2.44140625e-03,
        0.00000000e+00,  9.76562500e-03])

### 20.12

In [31]:
coord=np.array([0,0,
                600,0,
                1200,0]).reshape(3,2)

In [32]:
elcon=np.array([[0, 1],
                [1,2]])

In [33]:
bc_node = [0, 1, 4]

In [34]:
bc_val=[0., 0., 0.]

In [35]:
global_force=np.array([0,0,
                       0,-25000,
                      0,0])

In [36]:
FEA_u(coord, elcon, bc_node, bc_val, global_force, I, E)

array([ 0.00000000e+00,  0.00000000e+00, -1.75781250e-03, -1.46484375e-05,
        0.00000000e+00,  1.17187500e-05])

### 20.13

In [37]:
x=Symbol('x')
L=Symbol('L')

In [38]:
B2=diff(x-2*x**2/L+x**3/L**2, x, x)

In [39]:
B3=diff(3*x**2/L**2-2*x**3/L**3, x, x)

In [40]:
integrate(B2*B3, (x, 0, L))

-6/L**2